In [3]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np

import cv2
import glob
import time
import pickle

from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.model_selection import train_test_split

In [5]:
vehicle_n_nonvehicle = glob.glob('train/*/*/*.png')
vehicles = []
non_vehicles = []
all_vehicles =[]
all_non_vehicles = []

for v_and_nv in vehicle_n_nonvehicle:
    if 'nonvehicle' in v_and_nv:
        all_non_vehicles.append(v_and_nv)
    else:
        all_vehicles.append(v_and_nv)